In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
from scipy.sparse import csr_matrix

print("RESTAURANT RECOMMENDER SYSTEM")


df = pd.read_csv("D:\\intern\\Dataset .csv")
df = df.dropna()
print("Dataset is loaded")


df["Combined_Text"] = df["Locality"].astype(str) + " " + df["Cuisines"].astype(str)


tfidf = TfidfVectorizer(stop_words='english')
text_vectors = tfidf.fit_transform(df["Combined_Text"])


scaler = MinMaxScaler()
df[["Average Cost for two", "Votes"]] = scaler.fit_transform(df[["Average Cost for two", "Votes"]])


features = hstack([text_vectors, 
                   df[["Average Cost for two", "Votes"]].values])


def recommend_restaurants(user_input_text, user_cost, user_votes, top_n=5):
    user_text_vec = tfidf.transform([user_input_text])
    user_df = pd.DataFrame([[user_cost, user_votes]], columns=["Average Cost for two", "Votes"])
    user_scaled = scaler.transform(user_df)
    user_scaled_sparse = csr_matrix(user_scaled)
    user_vec = hstack([user_text_vec, user_scaled_sparse])
    
    similarity = cosine_similarity(user_vec, features).flatten()
    indices = similarity.argsort()[::-1][:top_n]
    
    return df.iloc[indices][["Restaurant Name", "Locality", "Cuisines", "Average Cost for two", "Votes", "Aggregate rating"]]


user_pref = "Kormangala Italian"
preferred_cost = 800
preferred_votes = 300

recommendations = recommend_restaurants(user_pref, preferred_cost, preferred_votes)

print("\nTop Recommended Restaurants:\n")
print(recommendations.to_string(index=False))

RESTAURANT RECOMMENDER SYSTEM
Dataset is loaded

Top Recommended Restaurants:

     Restaurant Name  Locality                       Cuisines  Average Cost for two    Votes  Aggregate rating
    Virgin Courtyard  Sector 7                        Italian              0.002750 0.074721               4.4
       Cafe Parmesan Sector 15                        Italian              0.001250 0.073075               4.5
             Rooftop Pitampura North Indian, Chinese, Italian              0.000750 0.002195               3.3
  Boheme Bar & Grill     Saket North Indian, Chinese, Italian              0.001625 0.010792               3.3
The Pasta Bar Veneto   MG Road                        Italian              0.001125 0.025151               3.6
